# Use Mdapy Efficiently

### The key point is re-using the neighborlist information.

In [1]:
import mdapy as mp
mp.__version__

'1.0.0a1'

### Build a Copper system with 4,000,000 atoms.

In [2]:
system = mp.build_crystal('Cu', 'fcc', 3.615, nx=100, ny=100, nz=100)

### Check the system information.

In [3]:
system

Atom Number: 4000000
Box information:
[[361.5   0.    0. ]
 [  0.  361.5   0. ]
 [  0.    0.  361.5]]
Origin: [0. 0. 0.]
Triclinic: False
Boundary: [1 1 1]
Particle Information:
shape: (4_000_000, 4)
┌──────────┬──────────┬──────────┬─────────┐
│ x        ┆ y        ┆ z        ┆ element │
│ ---      ┆ ---      ┆ ---      ┆ ---     │
│ f64      ┆ f64      ┆ f64      ┆ str     │
╞══════════╪══════════╪══════════╪═════════╡
│ 0.0      ┆ 0.0      ┆ 0.0      ┆ Cu      │
│ 0.0      ┆ 1.8075   ┆ 1.8075   ┆ Cu      │
│ 1.8075   ┆ 0.0      ┆ 1.8075   ┆ Cu      │
│ 1.8075   ┆ 1.8075   ┆ 0.0      ┆ Cu      │
│ 0.0      ┆ 0.0      ┆ 3.615    ┆ Cu      │
│ …        ┆ …        ┆ …        ┆ …       │
│ 359.6925 ┆ 359.6925 ┆ 354.27   ┆ Cu      │
│ 357.885  ┆ 357.885  ┆ 357.885  ┆ Cu      │
│ 357.885  ┆ 359.6925 ┆ 359.6925 ┆ Cu      │
│ 359.6925 ┆ 357.885  ┆ 359.6925 ┆ Cu      │
│ 359.6925 ┆ 359.6925 ┆ 357.885  ┆ Cu      │
└──────────┴──────────┴──────────┴─────────┘

### Direct calculation.

In [4]:
%%time
system.cal_centro_symmetry_parameter(12)
system.cal_ackland_jones_analysis()
system.cal_structure_entropy(rc=5.0, sigma=0.2)

CPU times: user 1min 59s, sys: 8.74 s, total: 2min 8s
Wall time: 12.5 s


In [5]:
system.data.head()

x,y,z,element,csp,aja,entropy
f64,f64,f64,str,f64,i32,f64
0.0,0.0,0.0,"""Cu""",3.2952e-26,1,-8.645166
0.0,1.8075,1.8075,"""Cu""",1.0984e-26,1,-8.645166
1.8075,0.0,1.8075,"""Cu""",1.0984e-26,1,-8.645166
1.8075,1.8075,0.0,"""Cu""",1.0984e-26,1,-8.645166
0.0,0.0,3.615,"""Cu""",2.1968e-26,1,-8.645166


### Re-use neighborlist information.

In [6]:
%%time
system.build_neighbor(rc=5.0, max_neigh=50) # Obtain the neighbor first, the following calculation can use it.
system.cal_structure_entropy(rc=5.0, sigma=0.2)
system.cal_ackland_jones_analysis()
system.cal_centro_symmetry_parameter(12)

CPU times: user 1min 19s, sys: 948 ms, total: 1min 20s
Wall time: 6.91 s


In [7]:
system.data.head()

x,y,z,element,csp,aja,entropy
f64,f64,f64,str,f64,i32,f64
0.0,0.0,0.0,"""Cu""",3.2952e-26,1,-8.645166
0.0,1.8075,1.8075,"""Cu""",1.0984e-26,1,-8.645166
1.8075,0.0,1.8075,"""Cu""",1.0984e-26,1,-8.645166
1.8075,1.8075,0.0,"""Cu""",1.0984e-26,1,-8.645166
0.0,0.0,3.615,"""Cu""",2.1968e-26,1,-8.645166
